# Neural TTR

This module is defined in the Python file nu.py

In [1]:
from pprint import pprint
from nu import Type, BType, PType, DepType, Pred, MeetType_n, FunType, InhibitType_n, StringType_n, Ty, iota, gensym_n, nu, and_n, MeetType, or_n, JoinType, labels, Rec, RecType
from neurons import Network, Neuron, Synapse, ActivityPattern
from utils import show, example

## Example 1:  `nu` maps types to types of neural activation.

Types in general correspond to a pattern of activation on a given network. We define a type `T` called `MyType`.  Its corresponding type of neural activation is `nu(T)` which by default has the name `MyType_n`.

In [2]:
T = Type('MyType')
Tn = nu(T)
print(show(Tn))



MyType_n


We now create a network `N` and add a single neuron `n1` to it.  A network is a one dimensional array of neurons. `n1` will be indexed as `[0]` in the network `N`.  We define an activity pattern `h1` which is a two dimensional array indicating which neurons should be activated at each step.  `h1` has only a single step which activates the 0th neuron in a network. We add this activity pattern to the type `Tn` we defined above.  Note that when we add an activity pattern for a type we mention the network for which it holds.  This type may be implemented as different patterns of activation for different networks.  That is, we should not expect to find identical implementations of types in different brains.

In order to see what happens in `N` when we it represents this type we use the method `ntrace` to turn on neural tracing.  We then create an instance of the neural type `Tn` on `N` and display `N`'s history.  The neuron has been given a default name of `neuron1`.  First it was off and then in was on.  That is, first there was no signal on its axon and then there was.

In [3]:
N = Network()
n1 = N.add_neuron()
h1 = ActivityPattern([[0]])
Tn.add_apat(N,h1)
N.ntrace()
Tn.create_n(N)
N.display_history()

-------  -  -
neuron0  0  1
-------  -  -


Neural types have a method `query_n` which takes a network as argument and tells you whether the history you have recorded using `ntrace` contains an instantiation of the neural type.

In [4]:
print(Tn.query_n(N))

True


## Example 2:  Activity patterns can involve more than one neuron

Here we create a neural type for a basic type. 

In [5]:
T = BType('MyBasicType')
Tn = nu(T)
print(show(Tn))


MyBasicType_n


In this example `T` corresponds to the activation of two neurons.

In [6]:
N = Network()
n1 = N.add_neuron()
n2 = N.add_neuron()
h1 = ActivityPattern([[0],[1]])
Tn.add_apat(N,h1)
N.ntrace()
Tn.create_n(N)
N.display_history()



-------  -  -  -
neuron1  0  1  1
neuron2  0  0  1
-------  -  -  -


And, as before, we can check whether the history instantiates the neural type.

In [7]:
print(Tn.query_n(N))

True


## Example 3:  neural types for recursive ptypes

In TTR 'hug($a$,$b$)' is known a ptype (a type constructed from a predicate together with its arguments).  Intuitively it is a type of situation in which $a$ hugs $b$.

Here we use straight pyttr to create a type `Ind`, with name `'Ind'`, to serve as the type of individuals and we use the `judge`-method to assert that `'a'` and `'b'` are of this type.  (We are allowing individuals to be represented by Python strings.)  We declare `hug` to be a predicate, with name `'hug'` which takes two arguments of type `Ind`.  We can now create a ptype with `hug` as predicate and `'a'` and `'b'` as arguments.  Here we set the Python variable `hug_a_b` to this ptype.  We can use `show` to give a not unexpected printout of this ptype.

In [8]:
Ind = Type('Ind')
Ind.judge('a')
Ind.judge('b')
hug = Pred('hug',[Ind,Ind])
hug_a_b = PType(hug,['a','b'])
print(show(hug_a_b))



hug(a, b)


We now connect what we have done to neural TTR, starting by creating a network. `iota` (think of $\iota$ as related to individuals) is a repository of activation patterns corresponding to individuals represented as Python strings.  `iota` has a method `add_grandmother` (as in  "grandmother neuron") which will add a single neuron to a network and associate activation of that neuron with the individual in its first argument. 

We create a type of neural activation, `hug_n`, corresponding to the predicate `hug` and call the `add_grandmother`-method for that type.  This will create a new neuron for the network and make activation of that neuron be the activity pattern associated with the type `hug_n`.  We now have what we need to create a neural type for `hug_a_b` and we set the Python variable `hug_a_b_n` to the neural type.

We now turn on neural tracing, create an event of type `hug_a_b_n` on the network and display the history.

In [9]:
N = Network()
iota.add_grandmother('a',N)
iota.add_grandmother('b',N)
hug_n = nu(hug)
hug_n.add_grandmother(N)
hug_a_b_n = nu(hug_a_b)

N.ntrace()
hug_a_b_n.create_n(N)
N.display_history()



------  -  -  -  -  -
a       0  0  1  0  0
b       0  0  0  1  0
hug_n   0  1  0  0  0
ptype2  2  1  1  1  0
rel     2  1  0  0  0
arg0    2  0  1  0  0
arg1    2  0  0  1  0
------  -  -  -  -  -


This trace of the history of the network is possibly more complicated than you had been expecting.  The first three rows correspond to the neurons that we added by our calls to the `add_grandmother` methods.  In the first column these three neurons are off (the network is at rest). Then in three successive steps the predicate, the first argument and the second argument are activated.  Finally, in the fifth column, the network is at rest again.  

Why did we choose to activate the neurons in order rather than, say, activate all three at the same time?  The reason is that the network needs to be able to distinguish between 'hug($a$,$b$)' and 'hug($b$,$a$)'.  If all three neurons were activated at the same time both of these would have the same representation.  In the literature on neural representation this is often referred to as $\textit{the binding problem}$.

But what is going on in the lower four rows of the history display?  In creating a neural event of the required type, the network discovered that it needed to create more neurons for book-keeping purposes.  In the first column these neurons were not present in the network and this is represented by the occurrences of `2`.  By the time we get to the second column the additional four neurons have been created.  Creating new neurons like this may seem implausible as a model of what happens in an animal brain.  However, it does not seem so implausible that many neurons in an animal brain are available to be designated as having a certain function.  Thus the creation of new neurons in our computational model could be seen as corresponding to the use of previously unused neurons in an animal brain.

The new neurons are used for bookkeeping purposes.  How do we know that an event on the network of the type represented in the top three rows is in fact the representation of a ptype rather than just a sequence of three representations: of a predicate, followed by a representation of `'a'` and a separate representation of `'b'`.  The three events are tied together by the activation of the neuron labelled `ptype2` ("ptype with two arguments") throughout the duration of the three events.  The role of the individual events in the ptype are indicated by the remaining three neurons, labelled `rel`, `arg0` and `arg1` which are active simultaneously with `hug_n`, `a` and `b` respectively.  This means that the order of the three individual events is of itself not important, although the current implementation will always produce them in the order given here.  They could, however, in principle occur in any order preserving the simultaneity of activation of the role-labelling neurons and the neurons `hug_n`, `a` and `b`.  This, then, represents our approach to the binding problem.

Another central problem for representing the kind of content you need for natural language on a neural architecture is known as $\textit{the recursion problem}$.  We illustrate this here by allowing predicates to take ptypes as arguments, thus allowing ptypes within ptypes to an arbitrary depth of embedding.

First we turn off neural tracing in the network `N`.  This will remove the previous history of the network and no events on the network will be stored until we restart neural tracing.

We now extend our environment using pyttr.  We declare a new individual `'c'` and a new predicate `believe` whose first argument is required to be an individual and whose second argument is required to be a type.  (The type `Ty` is provided by pyttr as the type of types, that is, anything defined as a type in pyttr will be of the type `Ty`.)  We then define the ptype `believe_c_hug_a_b` corresponding to believe($c$, hug($a$, $b$)). 

We now use neural TTR to create a type of neural activation `believe_c_hug_a_b_n` corresponding to this ptype. We add a grandmother neuron for `'c'` to `N`, generate the type of neurological activation `believe_n` for the predicate `believe` and add a grandmother neuron for it to the network.

Finally, we turn on neural tracing, create an event of type `believe_c_hug_a_b_n` on the network and display the history.

In [ ]:
N.nontrace()


Ind.judge('c')
believe = Pred('believe',[Ind,Ty])
believe_c_hug_a_b = PType(believe,['c',hug_a_b])

believe_c_hug_a_b_n = nu(believe_c_hug_a_b)
iota.add_grandmother('c',N)
believe_n = nu(believe)
believe_n.add_grandmother(N)


N.ntrace()
believe_c_hug_a_b_n.create_n(N)
N.display_history()


Note that we now have a second collection of neurons `ptype2`, `rel`, `arg0` and `arg1` to deal with the fact that we have represented a ptype which has a ptype within it.  Notice that the activation of `arg1` for the `ptype2` whose relation is represented by `believe_n` coincides with the activation of the second `ptype2` whose relation is represented by `hug_n`.

We can increase the embedding to an arbitrary depth, 'know($d$, believe($c$, hug($a$,$b$)))' and so on, and with each increase in depth we will generate a new collection of book-keeping neurons.  While for any snapshot of the neural architecture at a point in time it will have what is needed to process a certain finite depth of embedding, there is in principle no limit to the depth of embedding that it can process since it can always expand to account for an extra level of embedding.  In practice, of course, the depth of embedding is limited by the computational resources available (the size of the memory in the computer or the number of neurons in the brain).  This simple technique might provide us with one route into understanding how the plasticity of a brain allows it to devote resources to tasks on an as needed basis.

This, then, is our proposal for dealing with the recursion problem. Note that it relies on two aspects of our proposal:  firstly, that informational content is represented as $\textit{events}$ on a network rather than as architectural structure and secondly, that the network has $\textit{plasticity}$ in that it can expand or dedicate unused neurons to a particular task. 

## Example 4: dependent types as functions with arbitrary depth

Another place where you need recursion is when you characterize functions, since a function can return another function as result.  In this implementation we only treat functions which are dependent types, that is, they return a type or a function which is a dependent type.  The examples we will deal with here are $\lambda v\!:\!\textit{Ind}\ .\ \textrm{hug}(v,b)$ and $\lambda x\!:\!\textit{Ind}\ .\ \lambda y\!:\!\textit{Ind}\ .\ \textrm{hug}(x,y)$.

First we use pyttr to create a dependent type.

In [ ]:
T = DepType('v',Ind,PType(hug,['v','b']))
print(show(T))

Now we use neural TTR to turn off neural tracing on the network `N`.  We create a type of neural activity corresponding to `Ind` and add a grandmother for it on `N`.  We now create a type of neural activity `Tn` corresponding to the dependent type.

We now turn on neural tracing and create an event of the neural type on `N` and display `N`'s history.

In [ ]:
N.nontrace()
Ind_n = nu(Ind)
Ind_n.add_grandmother(N)
Tn = nu(T)

N.ntrace()
Tn.create_n(N)
N.display_history()

Note that we now have a collection of book-keeping neurons corresponding to the structure of the function which is the dependent type.  When the neuron `lambda` is active we are in the midst of an event representing a function.  In the first part of the function we represent its domain (`dom`) and its variable (`var`).  These neurons are active simultaneously with the neuron `Ind_n`, encoding the type of the domain of the function. The body of the function is coded by the neuron `rng` ("range") which is in fact a two place ptype, as coded by the onset of the first `ptype2` neuron being simultaneous with that of `rng`.  The ptype is coded as before except that the first argument is now indicated to be the variable which was introduced in the function.

One way to show the kind of activity associated with a neural type on a network is to create an event on the network in the way that we have been doing so far.  Sometimes, however, it is more convenient to inspect the activity pattern which is encoded on the neural type for that particular network.  This is what we do below.

In [ ]:
pprint(Tn.show_apat(N))



The activity pattern is displayed as a list of lists of triples.  The first element in each triple is the unique identifier of a neuron.  The second element is the convenient name we have assigned the neuron to make our displays human readable.  The third element is either `1` for activate the neuron or `0` for deactivate.  A list of triple thus represents the activations and deactivations which are to be carried out at one particular time step.  The list of lists represents the activations/deactivations to be carried out over a series of time steps.  Note that if a neuron is activated at one time step it will remain active until it is deactivated at a subsequent time step.  Thus in this pattern neuron `14` (`lambda`) is activated in the first time step and is not deactivated until the final time step.

In [ ]:
N.nontrace()
N.ntrace()
T= DepType('x',Ind,DepType('y',Ind,PType(hug,['x','y'])))
Tn = nu(T)
Tn.create_n(N)
N.display_history()
pprint(Tn.show_apat(N))

## Example 5

In [ ]:
Ppty = FunType(Ind,Ty)
every = Pred('every',[Ppty,Ppty])
every_n = nu(every)
N = Network()
every_n.add_grandmother(N)

dog = Pred('dog',[Ind])
dog_n = nu(dog)
dog_n.add_grandmother(N)

run = Pred('run',[Ind])
run_n = nu(run)
run_n.add_grandmother(N)

Ind_n.add_grandmother(N)
dog_ppty = DepType('x',Ind,PType(dog,['x']))
run_ppty = DepType('x',Ind,PType(run,['x']))

Tedr = PType(every,[dog_ppty,run_ppty])
Tedr_n = nu(Tedr)
N.ntrace()
Tedr_n.create_n(N)
N.display_history()
pprint(Tedr_n.show_apat(N))



N.nontrace()
m = N.memorize_type(Tedr_n,'every dog runs')
N.ntrace()
m.excite()
N.run()
N.display_history()


## Example 6

In [ ]:
N = Network()
Ind_n.add_grandmother(N)
iota.add_grandmother('a',N)
iota.add_grandmother('b',N)
a_n = nu('a')
pprint(Ind_n.show_apat(N))
pprint(a_n.show_apat(N))
pprint(Ind_n.judgmnt_type_n(a_n).show_apat(N))

m = N.memorize_judgmnt(Ind_n,a_n,'a:Ind')
N.ntrace()
m.excite()
N.run()
N.display_history()

## Example 7

In [ ]:
# uses variables from example 5
N = Network()
every_n.add_grandmother(N)
dog_n.add_grandmother(N)
run_n.add_grandmother(N)
Ind_n.add_grandmother(N)
T = PType(every,[dog_ppty,run_ppty])
T_n = nu(T)
iota.add_grandmother('e',N)
e_n = nu('e')
m = N.memorize_judgmnt(T_n,e_n, 'e:every(dog,run)')
N.ntrace()
m.excite()
N.run()
N.display_history()

## Example 8

In [ ]:
N = Network()
T1 = Type('T1')
T2 = Type('T2')
T1_n = nu(T1)
T2_n = nu(T2)
T1_n.add_grandmother(N)
T2_n.add_grandmother(N)
and_n.add_grandmother(N)
iota.add_grandmother('a',N)
T3 = MeetType(T1,T2)
T3_n = nu(T3)
m = N.memorize_judgmnt(T3_n,a_n,'a:T1&T2')
N.ntrace()
m.excite()
N.run()
N.display_history()

print(N.match_apat(T1_n.judgmnt_type_n(a_n).getapat(N)))
print(N.match_apat(T2_n.judgmnt_type_n(a_n).getapat(N)))


## Example 9

In [ ]:
N = Network()
T1 = Type('T1')
T2 = Type('T2')
T1_n = nu(T1)
T2_n = nu(T2)
T1_n.add_grandmother(N)
T2_n.add_grandmother(N)
or_n.add_grandmother(N)
iota.add_grandmother('a',N)
T3 = JoinType(T1,T2)
T3_n = nu(T3)
m = N.memorize_judgmnt(T3_n,a_n,'a:T1vT2')
N.ntrace()
m.excite()
N.run()
N.display_history()

print(N.match_apat(T1_n.judgmnt_type_n(a_n).getapat(N)))
print(N.match_apat(T2_n.judgmnt_type_n(a_n).getapat(N)))


## Example 10

In [ ]:
#Subtyping for neural types in terms of a relation on apats on a given network.  Works for these examples...
print(T1_n.judgmnt_type_n(a_n).subtype_of_n(T3_n.judgmnt_type_n(a_n),N))
and_n.add_grandmother(N)
T4 = MeetType(T1,T2)
T4_n = nu(T4)
print(T1_n.judgmnt_type_n(a_n).subtype_of_n(T4_n.judgmnt_type_n(a_n),N))


## Example 11

In [ ]:
N = Network()
labels.add_grandmother('l_x',N)
labels.add_grandmother('l_e',N) 
iota.add_grandmother('a',N)
iota.add_grandmother('s',N)
r = Rec({'l_x':'a','l_e':'s'})
r_n = nu(r)
pprint(r_n.show_apat(N))

N.ntrace()
r_n.create_n(N)
N.display_history()


## Example 12

In [ ]:
N = Network()
labels.add_grandmother('l_x',N)
labels.add_grandmother('l_e',N)
Ind_n.add_grandmother(N)
dog_n.add_grandmother(N)
Dog = DepType('v',Ind,PType(dog,['v']))
T_dog = RecType({'l_x':Ind,
                 'l_e':(Dog,['l_x'])})
T_dog_n = nu(T_dog)
pprint(T_dog_n.show_apat(N))

N.ntrace()
T_dog_n.create_n(N)
N.display_history()
#Problem with two labels at same time in dependent fields
#Now solved: a label neuron is marked as either a label or part of a value

#Random order? np.random.shuffle()


## Example 13

In [ ]:
#Function application
N = Network()
dog_n.add_grandmother(N)
Ind_n.add_grandmother(N)
Dog = DepType('v',Ind,PType(dog,['v']))
iota.add_grandmother('a',N)
print(show(Dog.app('a')))
print('\n')
Dog_n = nu(Dog)
a_n = nu('a')
Dog_a_n = nu(Dog.app('a'))
pprint(Dog_n.show_apat(N))
print('\n')
pprint(a_n.show_apat(N))
print('\n')
pprint(Dog_a_n.show_apat(N))


## Example 14

In [ ]:
#Substitution in records
N = Network()
labels.add_grandmother('l_x',N)
labels.add_grandmother('l_e',N) 
iota.add_grandmother('a',N)
iota.add_grandmother('s',N)
r = Rec({'l_x':'a','l_e':'s'})
iota.add_grandmother('s1',N)
r1 = r.subst('s','s1')
r1_n = nu(r1)
pprint(r1_n.show_apat(N))

## Example 15

In [ ]:
N = Network()
labels.add_grandmother('l_x',N)
labels.add_grandmother('l_e',N)
Ind_n.add_grandmother(N)
dog_n.add_grandmother(N)
cat = Pred('cat',[Ind])
cat_n = nu(cat)
cat_n.add_grandmother(N)
Dog = DepType('v',Ind,PType(dog,['v']))
Cat = DepType('v',Ind,PType(cat,['v']))
T_dog = RecType({'l_x':Ind,
                 'l_e':(Dog,['l_x'])})
T_cat = T_dog.subst(Dog,Cat)
print(show(T_dog))
print(show(T_cat))
T_dog_n = nu(T_dog)
T_cat_n = nu(T_cat)
pprint(T_dog_n.show_apat(N))
print('\n')
pprint(T_cat_n.show_apat(N))


## Example 16

In [ ]:
N = Network()
labels.add_grandmother('l_x',N)
labels.add_grandmother('l_e',N) 
iota.add_grandmother('a',N)
iota.add_grandmother('s',N)
Ind_n.add_grandmother(N)
r = Rec({'l_x':'a','l_e':'s'})
r_n = nu(r)
dog_n.add_grandmother(N)
Dog = DepType('v',Ind,PType(dog,['v']))
T_dog = RecType({'l_x':Ind,
                 'l_e':(Dog,['l_x'])})
T_dog_n = nu(T_dog)
pprint(T_dog_n.resolve(r_n).show_apat(N))
print('\n')
j_n = T_dog_n.judgmnt_type_n(r_n)
pprint(j_n.show_apat(N))
print('\n')



N.ntrace()
j_n.create_n(N)
N.display_history()



labels.add_grandmother('l_type',N)
labels.add_grandmother('l_obj',N)
j_n = nu(T_dog.aus_prop(r))
pprint(j_n.show_apat(N))
N.nontrace()
N.ntrace()
j_n.create_n(N)
N.display_history()

# See ausprop.pdf for an annotated version of the last example
